# Python Pizza - New Year's Party

# PyRT 2021 - Computer Graphics with Python

https://hamburg.python.pizza/

<img src="https://pbs.twimg.com/profile_images/1025003214029119489/juMkCCZm_400x400.jpg" width="12.5%"></img>


This notebook can be downloaded here: https://github.com/martinchristen/PythonPizzaNYE2020 

## About Me

Martin Christen, martin.christen@fhnw.ch<br/>
Professor of Geoinformatics and Computer Graphics

Twitter @MartinChristen


## What is PyRT ?

https://github.com/martinchristen/pyRT


* PyRT - The Python Raytracer.
* I originally developed pyRT for **teaching computer graphics**.
* One part of pyrt is the **virtual framebuffer** where you can draw Pixels using standard algorithms such as Bresenham's line drawing algorithm [1]
* From Version 0.5.0 an additional goal is better Jupyter integration, this is now done in RGBImage.
* **server side rendering** without depending on a graphics card.


[1]  J. E. Bresenham, 1965. Algorithm for computer control of a digital plotter. In: IBM Systems Journal, 4, 1, 25–30, ISSN 0018-8670<br/>

## Requirements

* This notebook requires the following modules: pyrt, pillow, numpy, and moviepy
* This notebook currently only runs on **Chrome Browser**. Yes, other browsers will be supported in 2021.


`pip install pyrt` <br/>
`pip install moviepy`<br/>
`conda install pillow numpy`


### Virtual Framebuffer for Pixel Operations


In [ ]:
from pyrt.renderer import RGBImage
from pyrt.math import Vec2, Vec3
import random

#### Animated Virtual Framebuffer in Jupyter


In [ ]:
w = 320
h = 240
image = RGBImage(w, h)
image.clear(Vec3(0.0,0.0,0.4))

In [ ]:
for i in range(5000):
    position = Vec2(random.randint(0, w - 1), random.randint(0, h - 1))
    color = Vec3(random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1))
   
    image.drawPoint(position, color, 1)

In [ ]:
image.framebuffer()

In [ ]:
for i in range(100):
    pos1 = Vec2(random.randint(0, w - 1), random.randint(0, h - 1))
    pos2 = Vec2(random.randint(0, w - 1), random.randint(0, h - 1))
    color = Vec3(random.uniform(0, 1), random.uniform(0, 1), random.uniform(0, 1))
   
    image.drawLine(pos1, pos2, color, 2)
    image.update(fps=30)

In [ ]:
type(image.data)

In [ ]:
print(image.data) # and yes, we could manipulate data directly here (or display with matplotlib)

#### Create a Movie

For more details check the moviepy docs: https://zulko.github.io/moviepy/ref/VideoClip/VideoClip.html

In [ ]:
import moviepy.editor as mpy

from IPython.display import Video
from IPython.display import Image

In [ ]:
w = 320
h = 240
image = RGBImage(w, h)

In [ ]:
def make_frame(t):
    image.drawCircleFilled(Vec2(127,127), int(t*8), Vec3(0,0,0), Vec3(1,0,0), 1)
    return image.data

In [ ]:
image.clear(Vec3(0.0,0.0,0.4))
clip = mpy.VideoClip(make_frame, duration=5)

In [ ]:
clip.write_videofile("movie.mp4", fps=30)

In [ ]:
Video("movie.mp4")

In [ ]:
image.clear(Vec3(0.0,0.0,0.4))
clip.write_gif("movie.gif",fps=30, opt="OptimizePlus", fuzz=10)

In [ ]:
Image("movie.gif")

#### Loading Images

In [ ]:
from pyrt.renderer import loadimage

image2 = loadimage("data/worldmap/world600.jpg")
image2.framebuffer("world")

In [ ]:
image2.drawCircleFilled(Vec2(300,150), radius=10, color=Vec3(1,0,0), fillcolor=Vec3(1,1,0), size=1)
image2.update("world")

In [ ]:
image2.drawCircleFilled(Vec2(100,50), radius=30, color=Vec3(0,0,0), fillcolor=Vec3(0,1,0), size=3)
image2.update("world")

# Animated Stars

In [ ]:
from pyrt.renderer import RGBImage
from pyrt.math import Vec2, Vec3
import random

w = 600
h = 400
image = RGBImage(w, h)
image.clear(Vec3(0.0,0.0,0.0))
image.framebuffer('stars')

**Create new View for this output**: right click on blue bar and select "Create New View for Output" and move it to top right (Jupyter Lab only)

In [ ]:
class Star:
    def __init__(self, pos,size,color):
        self.pos = pos # Vec2
        self.size = size # int
        self.color = color # Vec3

In [ ]:
# Create a list of 2000 stars:

stars = []
for i in range(0,2000):
    position = Vec2(random.randint(0, w-1), random.randint(0, h-1))
    s = random.randint(1,4)
    color = Vec3(s/4,s/4,s/4)
    newstar = Star(position,s,color)
    stars.append(newstar)

In [ ]:
def render_stars(stars):
    for star in stars:
        image.drawPoint(star.pos, star.color, star.size)
    image.update('stars', fps=30)

In [ ]:
render_stars(stars)

In [ ]:
def update_stars(stars):
    for star in stars:
        # increase position
        star.pos.x = star.pos.x + star.size*2
        if star.pos.x>=w:
            star.pos.x=0
            
    image.clear(Vec3(0.0,0.0,0.0))
    render_stars(stars)

In [ ]:
update_stars(stars)

In [ ]:
for i in range(0,250):
    update_stars(stars)

### Fireworks - Particle Explosion


We're creating two classes:

* **Particle**: a single particle with a position, velocity, speed, size, lifetime
* **Explosion**: a collection of particles with a velocity along a circle and a random speed, size, color


<img src="docs/img/circle.png" width="25%"/>


In [ ]:
import math, random

image.clear(Vec3(0.0,0.0,0.0))
image.update('stars')

In [ ]:
class Particle:
    def __init__(self, pos, velocity, color, size, frames=-1):
        self.pos = pos # position, Vec2
        self.v = velocity # velocity of particle, Vec2
        self.color = color # color of particle, Vec3
        self.size = size # size of particle, int
        self.frames = frames # number of frames before this particle is removed, int

In [ ]:
class Explosion:
    def __init__(self, pos, num_particles):
        self.particles = []
        self.maxframes = 100
        
        for i in range(0,num_particles):
            angle = 2*(0.5-random.random())*math.pi
            
            
            color = Vec3(0.8+random.random()*0.2,0.3*random.random(),0.3*random.random())
            s = random.randint(1,4)
    
            size = random.randint(1,3)
            speed = 3 # 3*random.random() # if constant -> circle
            p = Particle(pos, Vec2(math.cos(angle)*speed, 
                                   math.sin(angle)*speed), 
                                   color, size, self.maxframes)
            self.particles.append(p)
            
    def update(self):
        for p in self.particles:
            p.pos = Vec2(p.pos.x + p.v.x, p.pos.y + p.v.y)
            p.frames -= 1
            if p.frames < 0:
                p.frames = 0
       
            
    def draw(self):
        for p in self.particles:
            fade = p.frames / self.maxframes
            image.drawPoint(Vec2(int(p.pos.x), 
                                 int(p.pos.y)), 
                            fade * p.color, 
                            p.size)
   
        
        
        

In [ ]:
e = Explosion(Vec2(w//2,h//2), 500)

image.clear(Vec3(0.0,0.0,0.0))
for i in range(0,100):
    image.clear(Vec3(0.0,0.0,0.0))  
    e.update()
    e.draw()
    image.update('stars')
 

#### Multiple particle explosions

In [ ]:
e0 = Explosion(Vec2(w//2,h//2), 500)
for i in range(0,100):
    image.clear(Vec3(0.0,0.0,0.0))
    e0.update()
    e0.draw()
    image.update('stars')
    
e1 = Explosion(Vec2(w//4,h//3), 800)
for i in range(0,100):
    image.clear(Vec3(0.0,0.0,0.0))
    e1.update()
    e1.draw()
    image.update('stars')

    
e2 = Explosion(Vec2(w//2+30,h//3), 800)
e3 = Explosion(Vec2(w//4,h//4), 800)
for i in range(0,100):
    image.clear(Vec3(0.0,0.0,0.0))
    e2.update()
    e3.update()
    e2.draw()
    e3.draw()
    image.update('stars', fps=30)

### And now let's create a Movie

In [ ]:
e0 = Explosion(Vec2(w//2,h//2), 500)
e1 = Explosion(Vec2(w//4,h//3), 800)
e2 = Explosion(Vec2(w//2+30,h//3), 800)
e3 = Explosion(Vec2(w//4,h//4), 800)

# 30 fps x 10 seconds = 300 frames, so let's make 10 seconds
duration = 10
fps = 30

def make_frame(s):
    # s runs from 0...10 (if clip is 10 seconds)
    # calculate frame number:
    f = int(s*fps)
    
    if s < 3:
        image.clear(Vec3(0.0,0.0,0.0))
        e0.update()
        e0.draw()
    elif s < 6:
        image.clear(Vec3(0.0,0.0,0.0))
        e1.update()
        e1.draw()
    else:
        image.clear(Vec3(0.0,0.0,0.0))
        e2.update()
        e3.update()
        e2.draw()
        e3.draw()
    
    return image.data

In [ ]:
clip = mpy.VideoClip(make_frame, duration=duration)

In [ ]:
image.clear(Vec3(0.0,0.0,0.0))

In [ ]:
clip.write_gif("firework.gif",fps=fps, opt="OptimizePlus", fuzz=10)

In [ ]:
Image("firework.gif")

## Exercise

Now you know the basics about creating animations / movies with PyRT!

Create your own firework
   * better colors (changing colors?)
   * different animation type
   * create a gif animation out of it



<img src="https://2021.geopython.net/cfp.gif" width="50%"/>